In [8]:
import sys

In [9]:
import random

from distributed_processing.serializers import DummySerializer
from distributed_processing.client import Client
from distributed_processing.filesystem_connector import FileSystemConnector


In [10]:
DIRECTORY = 'G:/home/prueba_distribuida_fs'
NAMESPACE = "tasks"

In [11]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [12]:
fs_connector = FileSystemConnector(directory=DIRECTORY, namespace=NAMESPACE)

client = Client(DummySerializer(), fs_connector, check_registry="cache")

Client with id: redis_client_1
Results queue: redis_client_1_responses


In [13]:
fs_connector.methods_registry()

{'add': ['requests_cola_1'],
 'mul': ['requests_cola_1'],
 'div': ['requests_cola_1'],
 'eval_py_function': ['requests_py_eval']}

In [14]:
y = client.rpc_async("add", [1, 0])

In [15]:
y.get()

1

In [16]:
client.check_registry ="always"

try:
    y = client.rpc_async("fake", [1, 0])
except Exception as e:
    print(e)

'method_queues_fake'


In [17]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

try:
    y.get()
except Exception as e:
    print(e)


Error -32601 : The method does not exist/is not available.

 


In [18]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

y.safe_get(default="Esto es un error")

'Esto es un error'

In [19]:
client.check_registry ="cache"

In [20]:
x = client.rpc_async("div", [1, 0])

In [21]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32603 : Internal RPC error.

 Traceback (most recent call last):
  File "G:\python\distributed_processing\python\distributed_processing\worker.py", line 152, in _process_single_request
    result = func[request["method"]](*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\agarcia\AppData\Local\Temp\ipykernel_12760\1866336200.py", line 8, in div
    return x / y
           ~~^~~
ZeroDivisionError: division by zero



In [22]:
client.rpc_sync("add", [1, 1])

2

In [23]:
def f(x,y): return x + y

y = client.rpc_async_fn(f, [1, 2.0])

In [24]:
y.get()

3.0

In [25]:
client.rpc_sync_fn(f, [3.0, 2.0])

5.0

In [26]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)
    fs.append(client.rpc_async(t[0], t[1]))

('mul', [0.07284977345395194, 0.33457915029296403], {})
('div', [0.8435213207876749, 0.529404512660957], {})
('add', [0.26509239688356045, 0.028504495412920994], {})
('mul', [0.9680013233945953, 0.031346875835001176], {})
('div', [0.9588680661997471, 0.3072546004808958], {})
('add', [0.3937078171709585, 0.07458495896081985], {})
('div', [0.9088772237541215, 0.8903512705169007], {})
('mul', [0.4808217539107479, 0.5599169571920303], {})
('mul', [0.5713840330740967, 0.8393599224794034], {})
('mul', [0.7966565301123274, 0.7192718977789261], {})


In [27]:
[f.get() for f in fs]

[0.024374015301258167,
 1.5933398764356315,
 0.29359689229648145,
 0.030343817292567198,
 3.12076064833199,
 0.46829277613177833,
 1.0208074653798893,
 0.2692202534014411,
 0.47959685770704263,
 0.5730126542918679]

In [28]:
tp

[('mul', [0.07284977345395194, 0.33457915029296403], {}),
 ('div', [0.8435213207876749, 0.529404512660957], {}),
 ('add', [0.26509239688356045, 0.028504495412920994], {}),
 ('mul', [0.9680013233945953, 0.031346875835001176], {}),
 ('div', [0.9588680661997471, 0.3072546004808958], {}),
 ('add', [0.3937078171709585, 0.07458495896081985], {}),
 ('div', [0.9088772237541215, 0.8903512705169007], {}),
 ('mul', [0.4808217539107479, 0.5599169571920303], {}),
 ('mul', [0.5713840330740967, 0.8393599224794034], {}),
 ('mul', [0.7966565301123274, 0.7192718977789261], {})]

In [29]:
fs = client.rpc_batch_async(tp)

In [30]:
[f.get() for f in fs]

[0.024374015301258167,
 1.5933398764356315,
 0.29359689229648145,
 0.030343817292567198,
 3.12076064833199,
 0.46829277613177833,
 1.0208074653798893,
 0.2692202534014411,
 0.47959685770704263,
 0.5730126542918679]

In [31]:
client.rpc_batch_sync(tp)

[0.024374015301258167,
 1.5933398764356315,
 0.29359689229648145,
 0.030343817292567198,
 3.12076064833199,
 0.46829277613177833,
 1.0208074653798893,
 0.2692202534014411,
 0.47959685770704263,
 0.5730126542918679]

In [32]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "fake"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)

('mul', [0.8499289469786534, 0.03632374481764511], {})
('fake', [0.725250615108435, 0.5351738156407977], {})
('div', [0.06526901269081709, 0.6564031134704447], {})
('add', [0.11420207380494318, 0.6312388857553197], {})
('fake', [0.4940876558010089, 0.2666376858551993], {})
('mul', [0.2823006430244913, 0.7524879154535552], {})
('mul', [0.3915748235417875, 0.9252532509158119], {})
('add', [0.5571186581319969, 0.06738624895943524], {})
('add', [0.43174721953883566, 0.8589657662401902], {})
('mul', [0.8229174015784586, 0.9590364431448951], {})


In [33]:
tp

[('mul', [0.8499289469786534, 0.03632374481764511], {}),
 ('fake', [0.725250615108435, 0.5351738156407977], {}),
 ('div', [0.06526901269081709, 0.6564031134704447], {}),
 ('add', [0.11420207380494318, 0.6312388857553197], {}),
 ('fake', [0.4940876558010089, 0.2666376858551993], {}),
 ('mul', [0.2823006430244913, 0.7524879154535552], {}),
 ('mul', [0.3915748235417875, 0.9252532509158119], {}),
 ('add', [0.5571186581319969, 0.06738624895943524], {}),
 ('add', [0.43174721953883566, 0.8589657662401902], {}),
 ('mul', [0.8229174015784586, 0.9590364431448951], {})]

In [34]:
client.check_registry ="never"
client.set_default_queue("cola_1")

fs = client.rpc_batch_async(tp)

In [35]:
[f.safe_get() for f in fs]

[0.030872602183182427,
 None,
 0.09943434354803056,
 0.7454409595602629,
 None,
 0.21242782240069769,
 0.3623058784588243,
 0.6245049070914321,
 1.2907129857790258,
 0.7892077778118443]

In [36]:
client.check_registry ="cache"

In [37]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [38]:
try:    
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [39]:
client.check_registry="never"
client.set_default_queue("cola_1")

x = client.rpc_async("kk", [1, 0])

In [40]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 


In [41]:
y = client.rpc_async("add", [1, 0])

In [42]:
y.get(5)

1

In [43]:
def f(x,y): return x + y

client.check_registry="never"
y = client.rpc_async_fn(f, [1, 2.0])
try:
    y.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 
